### 웹 크롤링

#### requests

백그라운드에서 웹 크롤링을 진행.  

    requests.get("url")
    
입력한 URL을 호출함.

**<HTTP 응답 번호>**
- 200은 성공(205: ???)  
- 300번대는 리다이렉션 메시지  
- 400번대는 오류 메시지(403: 권한 없음, 404: 존재하지 않는 페이지)

웹페이지 개발자 도구 이용  
보통은 개발자 도구 페이지의 개체를 마우스 우클릭하면 나오는 XPath를 사용하여 코드 작성.

#### bs4

con.content

In [ ]:
import urllib as ulb
import requests as rqs
from bs4 import BeautifulSoup as bs

url = "https://ko.wikipedia.org/wiki/조선_세종"

a = rqs.get("http://www.naver.com")
b = bs(a.content, "lxml")

lst_kw = b.find_all(name = "a", class_ = "ah_k", attr = {"data-clk" : "lve.keyword"})
b
# lst_rk = b.fine_all(name = "span", class_ = "ah_r")

# for line in :
    # print(line.get_text())

파이썬으로 매일 오전 8시, 오후 8시 실시간 검색어를 7일간 크롤링하기.  
검색어의 순위 정보도 크롤링돼야 한다.

In [1]:
import urllib as ulb
import requests as rqs
from bs4 import BeautifulSoup as soup

In [4]:
date = ["2019-05-16", "2019-05-17", "2019-05-18", "2019-05-19", "2019-05-20", "2019-05-21", "2019-05-22", "2019-05-23", "2019-05-27"]
time = ["08%3A00%3A00", "20%3A00%3A00"]

url = "https://datalab.naver.com/keyword/realtimeList.naver?datetime=%sT%s&where=main" %(date[-1], time[0])
print(url)

rq_header = {
    "User-Agent" : ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.134 Safari/537.36 Vivaldi/2.5.1525.40")
}

page = rqs.get(url, headers = rq_header)
page.encoding = "utf-8"
print(page.url)
# print(page.history)

source_page = soup(page.content, "lxml")

https://datalab.naver.com/keyword/realtimeList.naver?datetime=2019-05-27T08%3A00%3A00&where=main
https://datalab.naver.com/keyword/realtimeList.naver?datetime=2019-05-27T08%3A00%3A00&where=main


In [11]:
def export_to_sql(d, t, k, r, c, s):
    dict_age = {"all" : "전체", "10s" : "10대", "20s" : "20대", "30s" : "30대", "40s" : "40대", "50s" : "50대 이상"}
    
    dict_time = {"08%3A00%3A00": "08:00", "20%3A00%3A00": "20:00"}
    
    sql = """INSERT INTO Search_Keywords(Date, Time, Keyword, Rank, Category, Source)
VALUES ('%s', '%s', '%s', '%s', '%s', '%s');""" %(d, dict_time[t], k, r, dict_age[c], s)
        
    return sql
    
    
def crawler(dates, times):
    rq_header = {
    "User-Agent" : ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.134 Safari/537.36 Vivaldi/2.5.1525.40")
    }
    
    ages = ["all", "10s", "20s", "30s", "40s", "50s"]
    
    for d in dates:  # 일자별로 키워드 수집
        for t in times:  # 시간별로 키워드 수집
            url = "https://datalab.naver.com/keyword/realtimeList.naver?datetime=%sT%s&where=main" %(d, t)
            
            page = rqs.get(url, headers = rq_header)
            page.encoding = "utf-8"
            
            source_page = soup(page.content, "lxml")
            
            for i in range(len(ages)):  # 나이별로 키워드 수집
                keywords = source_page.select(".section_lst_area.carousel_area.lst_narrow .keyword_rank div[data-age='%s'] span" %ages[i])
                
                kwds_txt = []
                for j in range(len(keywords)):  # 키워드(텍스트)만 추출하여 txt에 추가
                    kwds_txt.append(keywords[j].text)
                
                for j in range(len(kwds_txt)):
                    sql = export_to_sql(d, t, kwds_txt[j], j + 1, ages[i], "Naver")  # Date, Time, Keyword, Rank, Category, Source
                    # cursor.execute(sql)
    
    print("end")

    # cnx.commit()
    print("커밋됨.")

    # cnx.close()
    print("종료됨.")
    
    return

In [14]:
dates = ["2019-05-16", "2019-05-17", "2019-05-18", "2019-05-19", "2019-05-20", "2019-05-21", "2019-05-22", "2019-05-23"]

times = ["08%3A00%3A00", "20%3A00%3A00"]

crawler(["2019-05-23"], ["08%3A00%3A00"])

end
커밋됨.
종료됨.


0

In [5]:
# keywords = source_page.select(".section_lst_area.carousel_area.lst_narrow .keyword_rank > .rank_inner *")
# rank = source_page
keywords_all = source_page.select(".section_lst_area.carousel_area.lst_narrow .keyword_rank div[data-age='all'] span")

res_all = []
for i in range(len(keywords_all)):
    res_all.append(keywords_all[i].text)

res_all

['효린',
 '구하라',
 '위메프복권',
 '효린 학폭',
 '아스달 연대기',
 '효린 일진',
 '제주공항',
 '효린 카톡',
 '비트코인',
 '맨유 뮌헨',
 '봉준호 감독',
 '창원 날씨',
 '일기예보',
 '기온별 옷차림',
 '구해줘 홈즈',
 '티몬데이',
 '인천날씨',
 '날씨예보',
 '영화 기생충 줄거리',
 '김해날씨']

In [6]:
keywords_10s = source_page.select(".section_lst_area.carousel_area.lst_narrow .keyword_rank div[data-age='10s'] span")

res_10s = []
for i in range(len(keywords_10s)):
    res_10s.append(keywords_10s[i].text)

res_10s

['맨유 뮌헨',
 '구하라',
 '효린',
 '버스도착정보',
 '태풍',
 '에이틴',
 '2019 6월 모의고사',
 '맨유 레전드 매치',
 '기온별 옷차림',
 '빅히트샵',
 '진격의 거인 3기 17화',
 '김해날씨',
 '군산날씨',
 '인천날씨',
 '안산 날씨',
 '목포날씨',
 '익산 날씨',
 '원주 날씨',
 '순천날씨',
 '전주날씨']

In [7]:
keywords_20s = source_page.select(".section_lst_area.carousel_area.lst_narrow .keyword_rank div[data-age='20s'] span")

res_20s = []
for i in range(len(keywords_20s)):
    res_20s.append(keywords_20s[i].text)

res_20s

['구하라',
 '효린',
 '기온별 옷차림',
 '효린 카톡',
 '창원 날씨',
 '인천날씨',
 '위메프복권',
 '대탈출 시즌2',
 '아스달 연대기',
 '천안날씨',
 '수원날씨',
 '서울 날씨',
 '춘천 날씨',
 '부산 날씨',
 '청주 날씨',
 '안산 날씨',
 '제주공항',
 '김해날씨',
 '평택 날씨',
 '대전 날씨']

In [8]:
keywords_30s = source_page.select(".section_lst_area.carousel_area.lst_narrow .keyword_rank div[data-age='30s'] span")

res_30s = []
for i in range(len(keywords_30s)):
    res_30s.append(keywords_30s[i].text)

res_30s

['효린',
 '구하라',
 '위메프복권',
 '효린 카톡',
 '효린 일진',
 '아스달 연대기',
 '효린 학폭',
 '제주공항',
 '비트코인',
 '티몬데이',
 '구해줘 홈즈',
 '일기예보',
 '채지안',
 '네이버날씨',
 '신의탑',
 '이번주날씨',
 '제주날씨',
 '황금종려상',
 '기상청',
 '창원 날씨']

In [9]:
keywords_40s = source_page.select(".section_lst_area.carousel_area.lst_narrow .keyword_rank div[data-age='40s'] span")

res_40s = []
for i in range(len(keywords_40s)):
    res_40s.append(keywords_40s[i].text)

res_40s

['효린',
 '구하라',
 '위메프복권',
 '효린 학폭',
 '효린 일진',
 '아스달 연대기',
 '제주공항',
 '봉준호 감독',
 '효린 카톡',
 '영화 기생충 줄거리',
 '날씨예보',
 '구해줘 홈즈',
 '티몬데이',
 '비트코인',
 '기생충 개봉일',
 '씨스타',
 '케빈나',
 '로또860회당첨번호',
 '일기예보',
 '기상청']

In [10]:
keywords_50s = source_page.select(".section_lst_area.carousel_area.lst_narrow .keyword_rank div[data-age='50s'] span")

res_50s = []
for i in range(len(keywords_50s)):
    res_50s.append(keywords_50s[i].text)

res_50s

['구하라',
 '효린',
 '케빈나',
 '봉준호 감독',
 '효린 학폭',
 '영화 기생충 줄거리',
 '조진래',
 '송강호',
 '생일도',
 '송인택',
 '제주공항',
 '기생충 개봉일',
 '효린 일진',
 '나상욱',
 '로또860회당첨번호',
 '청년 전세자금대출',
 '날씨예보',
 'lpga실시간스코어',
 '파킹통장',
 '아스달 연대기']

In [10]:
import pymysql as pms

cnx = pms.connect(host = "plen-tutoring.mysql.database.azure.com", 
                  port = 3306, 
                  user = "plentir@plen-tutoring", 
                  password = "yjy990810!", 
                  database = "web_crawling", 
                  charset = "utf8")

cursor = cnx.cursor()

In [6]:
mk_table = """CREATE TABLE Search_Keywords(
ID INT UNSIGNED NOT NULL AUTO_INCREMENT, 
Date VARCHAR(10) NOT NULL, 
Time VARCHAR(8) NOT NULL, 
Keyword VARCHAR(255) NOT NULL, 
Rank INT NOT NULL, 
Category VARCHAR(20) NOT NULL, 
Source VARCHAR(50) NOT NULL, 
PRIMARY KEY(ID)
) DEFAULT CHARSET = utf8 AUTO_INCREMENT = 1;
"""

cursor.execute(mk_table)

0